In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from scipy.io.arff import loadarff 
import time
np.random.seed(42)

# Random Fourier Features (RFF) for Kernel Ridge Regression

## 1. Kernel Ridge Regression (KRR)
Kernel Ridge Regression solves the following optimization problem:

$$
\min_{\alpha} \|Y - K \alpha\|^2 + \lambda \|\alpha\|^2
$$

where:
- $Y$ is the target vector of shape $(n, 1)$,
- $K$ is the kernel matrix of shape $(n, n)$, defined by $K_{ij} = k(x_i, x_j)$,
- $\lambda$ is the regularization parameter,
- $\alpha$ is the vector of model coefficients.

The kernel function $k(x, z)$ for the Gaussian (RBF) kernel is defined as:

$$
k(x, z) = \exp\left(-\frac{\|x - z\|^2}{2 \sigma^2}\right)
$$

The solution to this problem is:

$$
\alpha = (K + \lambda I)^{-1} Y
$$

Given $\alpha$, predictions for a new input $x$ are:

$$
\hat{y} = \sum_{i=1}^n \alpha_i k(x, x_i)
$$

---

## 2. Motivation for Random Fourier Features
Computing the kernel matrix $K$ is expensive for large datasets, as it scales quadratically with the number of samples $n$. To approximate the kernel, we use Random Fourier Features, based on the following result:

### Bochner's Theorem
For a shift-invariant kernel $k(x, z) = k(x - z)$, there exists a probability distribution $p(\omega)$ such that:

$$
k(x, z) = \int_{\mathbb{R}^d} p(\omega) e^{j \omega^\top (x - z)} d\omega
$$

For the Gaussian kernel:

$$
k(x, z) = \exp\left(-\frac{\|x - z\|^2}{2 \sigma^2}\right)
$$

The corresponding distribution $p(\omega)$ is Gaussian:

$$
\omega \sim \mathcal{N}(0, \frac{1}{\sigma^2} I)
$$

---

## 3. Random Fourier Feature Approximation
We approximate the kernel by sampling $\omega$ from $p(\omega)$. Using Monte Carlo sampling, we can approximate $k(x, z)$ as:

$$
k(x, z) \approx \phi(x)^\top \phi(z)
$$

where $\phi(x)$ is the Random Fourier Feature mapping:

$$
\phi(x) = \sqrt{\frac{2}{D}} \cos(Wx + b)
$$

Here:
- $W$ is a matrix of random samples $\omega \sim \mathcal{N}(0, \frac{1}{\sigma^2} I)$,
- $b$ is a vector of random offsets $b \sim \text{Uniform}(0, 2\pi)$,
- $D$ is the number of random features.

---

## 4. Kernel Ridge Regression with RFF
Using the RFF mapping $\phi(x)$, we approximate the kernel matrix $K$ as:

$$
K \approx \Phi \Phi^\top
$$

where $\Phi$ is the feature matrix after applying $\phi(x)$ to all data points $X$.

### Modified Objective Function
We now rewrite the KRR optimization problem using $\Phi$:

$$
\min_w \|Y - \Phi w\|^2 + \lambda \|w\|^2
$$

where:
- $w$ is the weight vector in the transformed RFF space.

### Solution
The solution for $w$ is given by:

$$
w = (\Phi^\top \Phi + \lambda I)^{-1} \Phi^\top Y
$$

### Prediction
For a new input $x'$, the prediction is:

$$
\hat{y} = \phi(x')^\top w
$$

---

## 5. Advantages of RFF
1. **Efficiency**:
   - Avoids computing and storing the $n \times n$ kernel matrix.
   - Instead, transforms the data into an $n \times D$ feature matrix, where $D$ is typically much smaller than $n$.

2. **Scalability**:
   - Linear in $n$ for training and predictions.

3. **Flexibility**:
   - Can use any linear solver (e.g., Ridge Regression) on the transformed features.

---


In [ ]:
arff_file = loadarff('/home/scott/CSC2515_NCSI/Kernel/dataset/MagicTelescope.arff')
data = pd.DataFrame(arff_file[0])
data['labels'] = (data['class:'] == b'g').astype(int)
labels = data['labels'].to_numpy()
features = data.iloc[:,1:-2].to_numpy()

scaler = StandardScaler()
features = scaler.fit_transform(features)

In [ ]:
class RandomFourierFeatures:
    def __init__(self, input_dim, num_features, sigma):
        """
        Initialize the RFF transformer.
        
        Parameters:
        - input_dim: Dimensionality of the input features.
        - num_features: Number of random Fourier features (D).
        - sigma: Kernel bandwidth (scale parameter for Gaussian kernel).
        """
        self.num_features = num_features
        self.sigma = sigma
        self.W = np.random.normal(loc=0, scale=1/sigma, size=(input_dim, num_features))
        self.b = np.random.uniform(0, 2 * np.pi, size=num_features)
    
    def transform(self, X):
        """
        Transform the input features to the RFF feature space.
        
        Parameters:
        - X: Input features, shape (n_samples, input_dim).
        
        Returns:
        - Transformed features, shape (n_samples, num_features).
        """
        projection = X @ self.W + self.b
        return np.sqrt(2 / self.num_features) * np.cos(projection)


In [ ]:
def KRR_with_RFF(X_train_rff, y_train, lam, num_features=500):
    """
    Kernel Ridge Regression using Random Fourier Features (RFF).
    
    Parameters:
    - X_train: Training data, shape (n_train, input_dim).
    - y_train: Training labels, shape (n_train,).
    - X_test: Test data, shape (n_test, input_dim).
    - lam: Regularization parameter (lambda).
    - sigma: Bandwidth for the Gaussian kernel.
    - num_features: Number of random Fourier features to generate.
    
    Returns:
    - Predictions on the test data.
    """
    
    # Ridge regression in transformed space
    K = X_train_rff.T @ X_train_rff + lam * np.eye(num_features)
    try:
        alphas = np.linalg.solve(K, X_train_rff.T @ y_train)
    except:
        return 'Solve failed, check pairwise distances'
    else:
        return alphas

def KRR(X_train, y_train, lam, sigma):
    """
    Kernel Ridge Regression using Random Fourier Features (RFF).
    
    Parameters:
    - X_train: Training data, shape (n_train, input_dim).
    - y_train: Training labels, shape (n_train,).
    - X_test: Test data, shape (n_test, input_dim).
    - lam: Regularization parameter (lambda).
    - sigma: Bandwidth for the Gaussian kernel.
    - num_features: Number of random Fourier features to generate.
    
    Returns:
    - Predictions on the test data.
    """
    dist = pairwise_distances(X_train)
    exponent = dist/sigma

    K = np.exp(-(exponent**2)/2)
    K += np.eye(K.shape[0])*lam
    
    try:
        alphas = np.linalg.solve(K, y_train)
    except:
        return 'Solve failed, check pairwise distances'
    else:
        return alphas

def GridSearchCV(X, Y, params, num_features, cv=4, train_set_ratio=1):
    kf = KFold(n_splits=cv)
    X_train, X_test = [], []
    Y_train, Y_test = [], []
    for train, test in kf.split(X):
        train_size = int(train_set_ratio*len(train))
        sampled_train = np.random.choice(train, train_size, replace=False)
        X_train.append(X[sampled_train])
        Y_train.append(Y[sampled_train])
        X_test.append(X[test])
        Y_test.append(Y[test])

    best_score = 0  # Maximizing accuracy
    mat = np.zeros((len(params['lambda']), len(params['length'])))
    for i, lam in enumerate(params['lambda']):
        for j, sigma in enumerate(params['length']):
            accuracies = []
            for k in range(cv):
                rff = RandomFourierFeatures(input_dim=X_train[k].shape[1], num_features=num_features, sigma=sigma)

                #alphas = KRR(X_train[k], Y_train[k], lam, sigma)
                alphas = KRR_with_RFF(rff.transform(X_train[k]), Y_train[k], lam, num_features)

                if type(alphas) == str:
                    accuracy = 0
                else:
                    # normal KRR
                    #dist = scipy.spatial.distance_matrix(X_train[k], X_test[k])
                    #exponent = dist/sigma
                #
                    #K_test = np.exp(-(exponent**2)/2)
                    #preds = (K_test.T @ alphas) > 0.5
                    #accuracy = accuracy_score(preds,  Y_test[k])

                    # RFF
                    X_test_rff = rff.transform(X_test[k])
                    preds = (X_test_rff @ alphas) > 0.5
                    accuracy = accuracy_score(preds, Y_test[k])

                accuracies.append(accuracy)
            mat[i][j] = np.mean(accuracies)
            if np.mean(accuracies) > best_score:
                best_score = np.mean(accuracies)
                best_lambda = lam
                best_sigma = sigma
    
    #plt.imshow(mat, cmap="viridis", interpolation="nearest")
    #plt.colorbar(label="Accuracy")
    #plt.xlabel("$\sigma$", fontsize=14)
    #plt.ylabel("$\lambda$", fontsize=14)
    #plt.xticks(np.arange(len(params['length'])), [f'{sigma:.2f}' for sigma in params['length']], rotation=45)
    #plt.yticks(np.arange(len(params['lambda'])), [f'{lam:.2e}' for lam in params['lambda']])
    #plt.title(f"Grid Search Accuracy Heatmap", fontsize=16)
    #plt.show()

    return {'accuracy': best_score, 'lambda': best_lambda, 'length': best_sigma}

In [ ]:
training_fractions = [0.05, 0.1, 0.2, 0.4, 0.8]
test_size = int((1-max(training_fractions))*len(features))

num_features = 1

N = 10
all_accuracies = []
all_kernel_inversions = []
all_pred_times = []

for n in range(N):
    indices = list(range(len(features)))
    np.random.shuffle(indices)
    
    X = features[indices]
    Y = labels[indices]

    X_test = X[-test_size:]
    Y_test = Y[-test_size:]

    X_trains, Y_trains, = [], []

    for frac in training_fractions:
        training_size = int(frac*len(X))
        X_trains.append(X[:training_size])
        Y_trains.append(Y[:training_size])

    accuracies = []
    kernel_inversions = []
    pred_times = []
    
    for i in range(len(training_fractions)):
        print(f'Training size {training_fractions[i]*len(X)}')
        X_train = X_trains[i]
        Y_train = Y_trains[i]

        param_grid = {'length': np.logspace(-4,4, num=10),
                    'lambda': np.logspace(-12, 0, num=10)}
        best_params = GridSearchCV(X_train, Y_train, param_grid, num_features, 4, 1)
        print(best_params)

        rff = RandomFourierFeatures(input_dim=X_train.shape[1], num_features=num_features, sigma=best_params['length'])

        begin = time.time()
        #alphas = KRR(X_train, Y_train, best_params['lambda'], best_params['length'])
        alphas = KRR_with_RFF(rff.transform(X_train), Y_train, best_params['lambda'], num_features)
        end = time.time()
        kernel_inversion = end - begin

        # normal KRR
        #begin2 = time.time()
        #dist = scipy.spatial.distance_matrix(X_test,X_train)
        #exponent = dist/best_params['length']
        #K_test = np.exp(-(exponent**2)/2)
        #preds = np.dot(K_test,alphas)>0.5
        #end2 = time.time()
        #pred_time = end2 - begin2

        #accuracies.append(accuracy_score(preds, Y_test))
        #kernel_inversions.append(kernel_inversion)
        #pred_times.append(pred_time)
        # RFF
        begin2 = time.time()
        rff = RandomFourierFeatures(input_dim=X_train.shape[1], num_features=num_features, sigma=best_params['length'])
        X_test_rff = rff.transform(X_test)
        preds = X_test_rff @ alphas
        end2 = time.time()
        pred_time = end2 - begin2
        accuracies.append(accuracy_score(preds>0.5, Y_test))
        kernel_inversions.append(kernel_inversion)
        pred_times.append(pred_time)
        print(f'Kernel inversion took {kernel_inversion}, predictions {pred_time}')                                                                                       
        
    all_accuracies.append(accuracies)
    all_kernel_inversions.append(kernel_inversions)
    all_pred_times.append(pred_times)